<a href="https://colab.research.google.com/github/MariaReneeGonzalezBravo/SIMULACION-1/blob/main/Inventario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Sistema de inventarios**

*   27/07/24
*   González Bravo María Renée



La demanda mensual de un cierto producto sigue la siguiente distribución de probabilidad empírica:




In [318]:
import random as rd
import matplotlib.pyplot as plt
import numpy as np
from prettytable import PrettyTable as pt

In [332]:
# Listas de cantidades y probabilidades
c = [35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]
p = [0.010,0.015,0.020,0.020,0.022,0.023,0.025,0.027,0.028,0.029,0.035,0.045,
     0.060,0.065,0.070,0.080,0.075,0.070,0.065,0.060,0.050,0.040,0.030,0.016,0.015,0.005]

# tabla
table = pt()
table.field_names = ["Cantidad", "Probabilidad"]


for i in range(len(c)):
    table.add_row([c[i], p[i]])

print("Distribución de probabilidad empírica")
print(table)

Distribución de probabilidad empírica
+----------+--------------+
| Cantidad | Probabilidad |
+----------+--------------+
|    35    |     0.01     |
|    36    |    0.015     |
|    37    |     0.02     |
|    38    |     0.02     |
|    39    |    0.022     |
|    40    |    0.023     |
|    41    |    0.025     |
|    42    |    0.027     |
|    43    |    0.028     |
|    44    |    0.029     |
|    45    |    0.035     |
|    46    |    0.045     |
|    47    |     0.06     |
|    48    |    0.065     |
|    49    |     0.07     |
|    50    |     0.08     |
|    51    |    0.075     |
|    52    |     0.07     |
|    53    |    0.065     |
|    54    |     0.06     |
|    55    |     0.05     |
|    56    |     0.04     |
|    57    |     0.03     |
|    58    |    0.016     |
|    59    |    0.015     |
|    60    |    0.005     |
+----------+--------------+


El tiempo de entrega está distribuido de acuerdo a la siguiente función de probabilidad:


In [333]:
meses = [1,2,3]
p_entrega = [0.3,0.4,0.3]

table = pt()
table.field_names = ["Meses", "Probabilidad"]
for i in range(len(meses)):
    table.add_row([meses[i], p_entrega[i]])

print("Tiempo de entrega")
print(table)

Tiempo de entrega
+-------+--------------+
| Meses | Probabilidad |
+-------+--------------+
|   1   |     0.3      |
|   2   |     0.4      |
|   3   |     0.3      |
+-------+--------------+


Los factores estacionales para cada uno de los meses del año son como se muestra a continuación:


In [334]:
fe = [1.2, 1.0, 0.9, 0.8, 0.8, 0.7, 0.8, 0.9, 1.0, 1.2, 1.3, 1.4]

table = pt()
indices = np.arange(1, len(fe) + 1)
table.add_column("Mes", indices)
table.add_column("Factores estacionales", fe)

print("Factores estacionales")

print(table)


Factores estacionales
+-----+-----------------------+
| Mes | Factores estacionales |
+-----+-----------------------+
|  1  |          1.2          |
|  2  |          1.0          |
|  3  |          0.9          |
|  4  |          0.8          |
|  5  |          0.8          |
|  6  |          0.7          |
|  7  |          0.8          |
|  8  |          0.9          |
|  9  |          1.0          |
|  10 |          1.2          |
|  11 |          1.3          |
|  12 |          1.4          |
+-----+-----------------------+


Donde se tiene la siguiente información relevante respecto a los costos:

**Costo de ordenar** = $ \$100$/orden

**Costo de inventario** = $ \$20$/unidad por año

**Costo de faltante** = $ \$50$/unidad

Si el inventario inicial se asume que son 150 unidades, determine la catidad optima a ordenar (q) y el nivel optimo del reorden (R)

In [322]:
#acumulada de la lista de probabilidad
L_acum = []
acum = 0
for i in p:
    acum += i
    L_acum.append(acum)

#Se devuelve un valor dependiendo de la lista acumulada
def acum_i(u1=0.5):  #se define la función
  for i in range(len(c)):
    if u1 < L_acum[i]:
      return c[0]
    elif L_acum[i] <= u1 < L_acum[i+1]:
      return c[i+1]

In [323]:
L_p_entrega = []
acum = 0
for i in p_entrega:
    acum += i
    L_p_entrega.append(acum)

In [324]:
#Funcion que nos devuleve un tiempo de entrega respecto a su probabilidad
def p_i(u2 = 0.5):
  for i in range(len(meses)):
    if u2 < L_p_entrega[i]:
      return meses[0]
    elif  L_p_entrega[i] <= u2 < L_p_entrega[i + 1]:
      return meses[i + 1]

In [325]:
# Variables de control
pp = False #pedido pendiente
tpp = 0 #tiempo pedido pendiente
t_o = 0 # total de ordenes
ft = 0 #falta total
t_IMP = 0 #Tiempo IMP

#valores
n = 200 # Número de unidades a ordenar
i_0 = 150 # inventario inicial
r = 100 #reorden

**Simulacion Inventario**

In [326]:
#tabla
Tabla = pt(['Mes', 'Inventario inicial', 'Número aleatorio', 'Demanda Ajustada', 'Inventario final', 'Faltante', 'Orden', 'Inventario mensual promedio'])

for i in range(12):
    aux = i_0
    x = rd.uniform(0, 1)
    da= int(acum_i(x) * fe[i])
    inventario_final = i_0 - da

    if inventario_final > 0:
      imp = (aux + inventario_final) / 2 #inventario mensual promedio
      t_IMP += imp
    else:
      imp = round((aux / 2) * (aux / da))
      t_IMP += imp

    if inventario_final >= 0:
        f = 0 #valor faltante
    else:
        f = abs(inventario_final)
        ft += f
        inventario_final = 0



    # Control del pedido pendiente
    if pp:
        tpp -= 1
        if tpp == 0:
            inventario_final += n
            pp = False

    # Revisión de la necesidad de reordenar
    if inventario_final < r and not pp:
        x1 = rd.uniform(0, 1) #nuevo aleatorio
        tpp = p_i(x1)
        pp = True
        orden = 1
        t_o += 1
    else:
        orden = 0



    # tabla
    lista_tabla = [i + 1, aux, x, da, inventario_final, f, orden, imp]
    Tabla.add_row(lista_tabla)

    i_0 = inventario_final

#tabla final
print(Tabla)
print("Total de ordenes:", t_o)
print("Falta total:", ft)
print("Total de Imp:", t_IMP)

#calculo de cda costo y del inventario
Ci = t_IMP*1.67 #costo de llevar inventario
Cf = ft*50 #costo faltante
Co = t_o*100 # costo de ordenar
Ct = Ci + Cf + Co # costo total
print(' Costo total de ordenes: ',Co)
print(' Costo total de llevar inventario: ',Ci)
print(' Costo total de faltante: ',Cf)
print(' Costo total: ',Ct)

+-----+--------------------+---------------------+------------------+------------------+----------+-------+-----------------------------+
| Mes | Inventario inicial |   Número aleatorio  | Demanda Ajustada | Inventario final | Faltante | Orden | Inventario mensual promedio |
+-----+--------------------+---------------------+------------------+------------------+----------+-------+-----------------------------+
|  1  |        150         |  0.7093482271392009 |        62        |        88        |    0     |   1   |            119.0            |
|  2  |         88         |  0.8534213464363779 |        55        |        33        |    0     |   0   |             60.5            |
|  3  |         33         | 0.05025823735582269 |        34        |       200        |    1     |   0   |              16             |
|  4  |        200         |  0.6984082590472612 |        41        |       159        |    0     |   0   |            179.5            |
|  5  |        159         | 0.256

**Simulación repetida**

Para poder calcular la media del costo. Para esto definimos una función llamada "sim" donde sus parametros son las unidades a ordenar, el reorden que se va a generar y el número de iteraciones necesarias

In [327]:
import random as rd

def sim(No_de_unidades_a_ordenar, reorden, n=50):
    L1 = []
    L2 = []
    L3 = []
    L4 = []
    SUM1 = 0
    SUM2 = 0
    SUM3 = 0
    SUM4 = 0
    t = 1

    for i in range(n):
        i_0 = No_de_unidades_a_ordenar  # Inicializar i_0 al inicio de cada iteración de i
        t_IMP = 0
        ft = 0
        t_o = 0
        pp = False
        tpp = 0

        for j in range(12):
            aux = i_0
            n = rd.uniform(0, 1)
            da = int(acum_i(n) * fe[j])
            inventario_final = i_0 - da

            if inventario_final > 0:
                imp = (aux + inventario_final) / 2
                t_IMP += imp
            else:
                imp = round((aux / 2) * (aux / da))
                t_IMP += imp

            if inventario_final >= 0:
                f = 0
            else:
                f = abs(inventario_final)
                ft += f
                inventario_final = 0

            # Control del pedido pendiente
            if pp:
                tpp -= 1
                if tpp == 0:
                    inventario_final += n
                    pp = False

            # Revisión de la necesidad de reordenar
            if inventario_final < reorden and not pp:
                x1 = rd.uniform(0, 1)
                tpp = p_i(x1)
                pp = True
                orden = 1
                t_o += 1
            else:
                orden = 0

            i_0 = inventario_final

        Ci = t_IMP * 1.67
        Cf = ft * 50
        Co = t_o * 100
        Ct = Ci + Cf + Co

        SUM1 += Co
        SUM2 += Ci
        SUM3 += Cf
        SUM4 += Ct

        L1.append(SUM1 / t)
        L2.append(SUM2 / t)
        L3.append(SUM3 / t)
        L4.append(SUM4 / t)
        t += 1

    return L1[-1], L2[-1], L3[-1], L4[-1]

# Ejemplo de ejecución de la simulación
LIS1, LIS2, LIS3, LIS4 = sim(200, 100, 5000)

# Imprimir valores promedio de costos
print("Costo de ordenar promedio:", LIS1)
print("Costo de llevar inventario promedio:", LIS2)
print("Costo de faltante promedio:", LIS3)
print("Costo total promedio:", LIS4)


Costo de ordenar promedio: 569.14
Costo de llevar inventario promedio: 656.6661398517267
Costo de faltante promedio: 18959.95423006047
Costo total promedio: 20185.76036991219


In [328]:
# Ejecutar simulación
LIS1, LIS2, LIS3, LIS4 = sim(200,100,5000)
#imprimir valores
print("Costo de ordenar promedio:", LIS1)
print("Costo de llevar inventario promedio:", LIS2)
print("Costo faltante promedio:", LIS3)
print("Costo total promedio:", LIS4)

Costo de ordenar promedio: 571.32
Costo de llevar inventario promedio: 657.2298618973614
Costo faltante promedio: 18967.67537352541
Costo total promedio: 20196.22523542282


**OPTIMIZACIÓN**

Hacemos uso de el método de optimización Hooke-Jeeves, donde:

El método de optimización Hooke-Jeeves es un algoritmo iterativo que busca minimizar una función objetivo. Funciona explorando el vecindario de una solución actual, moviéndose en direcciones específicas (positivas y negativas) de cada parámetro. Si encuentra una mejora, ajusta la solución hacia esa dirección. Utiliza un criterio de convergencia para detenerse cuando los cambios son pequeños. Es efectivo para problemas donde no se dispone de información sobre la estructura analítica de la función objetivo y se basa en la búsqueda directa en el espacio de parámetros.




In [329]:
def hj(f, x0, delta=0.5, epsilon=1e-4, max_iter=100):
    n = len(x0)
    x = np.array(x0)
    xp = np.array(x0)
    xb = np.array(x0)
    for i in range(max_iter):
        for j in range(n):
            d = np.zeros(n)
            d[j] = delta
            if f(xb + d) < f(xb):
                xb = xb + d
            elif f(xb - d) < f(xb):
                xb = xb - d
        if np.linalg.norm(xb - xp) < epsilon:
            break
        xp = xb + (xb - xp)
        if f(xp) < f(xb):
            xb = xp
        x = xb
    return xb

# Fcosto total para optimizar
def ct(params):
    n, r = params
    _, _, _, Ct = sim(n, r,1000)
    return Ct

# Parâmetros iniciales
x0 = [200, 100] # [n, r]

# óptimo
opt = hj(ct, x0)
no, ro = opt # numero de unidades a ordenar valor optimo, reorden optimo

print(f"Óptimo de número de unidades a ordenar: {no}")
print(f"Óptimo del reorden: {ro}")

# Calcular los costos con los valores óptimos
co, cio, cfo, cto = sim(no, ro)
print(f"Costo a ordenar óptimo: {co}")
print(f"Costo de llevar el inventario óptimo: {cio}")
print(f"Costo faltante óptimo: {cfo}")
print(f"Costo total óptimo: {cto}")

Óptimo de número de unidades a ordenar: 215.0
Óptimo del reorden: 104.0
Costo a ordenar óptimo: 552.0
Costo de llevar el inventario óptimo: 754.1081669709197
Costo faltante óptimo: 18393.287613116307
Costo total óptimo: 19699.39578008722
